In [14]:
import pymongo
from datetime import datetime
import feedparser
from pymongo import MongoClient
import hashlib


In [72]:
import hashlib
import feedparser
from pymongo import MongoClient

def fetch_rss():
    # BD
    client = MongoClient('mongodb://localhost:27017/')
    db = client['BD']
    coleccion = db['Furbo']
    
    # pagina para robar datos
    feed = feedparser.parse("https://e00-marca.uecdn.es/rss/futbol/primera-division.xml")
    articles = []
    
    for entry in feed.entries:
        # para obtener las categorias
        categories = [category for category in entry.get('tags', [])]
        
        # Calcular el hash del artículo
        article_hash = hashlib.sha256(entry.title.encode('utf-8')).hexdigest()
        
        # Comprobar si el artículo ya existe en la base de datos usando el hash
        if (any(category.term for category in categories)) and coleccion.find_one({'hash': article_hash}) is None:
            article = {
                "hash": article_hash,
                "title": entry.title,
                "description": entry.link,
                "categories": [category.term for category in categories],  # Lista de todas las categorías
            }
            # inserta el articulo en la lista 
            articles.append(article)
            # guardarlo en la base de datos 
            coleccion.insert_one(article)
        
    return articles


In [74]:
furbo = fetch_rss()
furbo

[]

In [75]:
# imprimir nombre de la coleccion
print(db.list_collection_names())

# comprobar lo q hay en la coleccion
for doc in coleccion.find():
    print(doc)

['Furbo']
{'_id': ObjectId('67d1d1174cc04551e56eb7c8'), 'hash': 'bc9cc7e600f93345ac08bda8fe8e96ee90147d619464553944f7c90b28249f4c', 'title': 'Del Nido Benavente y la postura de los \'americanos\': "Sabemos muchas cosas y no sabemos nada"', 'description': 'https://www.marca.com/futbol/sevilla/2025/03/12/nido-benavente-postura-americanos-sabemos-muchas-cosas-sabemos.html', 'categories': ['Primera División', 'Sevilla FC', 'José María del Nido']}
{'_id': ObjectId('67d1d1174cc04551e56eb7c9'), 'hash': '57370d43034c433f67cfd1a20683cc894ae67cea84887b5def384eda198a298c', 'title': 'Parejo: “El Roig Arena es espectacular; el nuevo Mestalla no sé si lo voy a ver”', 'description': 'https://www.marca.com/futbol/villarreal/2025/03/12/parejo-roig-arena-espectacular-nuevo-mestalla-ver.html', 'categories': ['Primera División', 'Valencia CF', 'Villarreal CF']}
{'_id': ObjectId('67d1d1174cc04551e56eb7ca'), 'hash': 'd8a60f60455ab7d413cc9b5cc42508b14c2f0c248d3b41651b914ed2c905e628', 'title': 'El Roig Arena 

In [80]:
# filtrar por los que son de liga
articulos_la_liga = coleccion.find({"categories": "La Liga"})
articulos_la_liga


In [81]:
for u in articulos_la_liga:
    print(u)

{'_id': ObjectId('67d1d1174cc04551e56eb7cc'), 'hash': 'a2a4240a4fae9d662b2bbcd3fbbf8fd7e57ae2b842b79ab23ef1efb1490bef2c', 'title': 'Onces probables y chollos para la jornada 28 de LaLiga', 'description': 'https://www.marca.com/fantasy-marca/2025/03/12/67d09c4046163f46138b459a.html', 'categories': ['Fantasy MARCA', 'Primera División', 'La Liga', 'Liga Española', 'Fútbol']}
{'_id': ObjectId('67d1d1174cc04551e56eb7e4'), 'hash': 'b6bd99bf8b60e9574db1a1ddef429f9796a4789b83e30d7ab7abfb6af8afd90a', 'title': "El 'gasto en defensa' merece la pena", 'description': 'https://www.marca.com/fantasy-marca/2025/03/11/67d068ea22601d3e588b4584.html', 'categories': ['Fantasy MARCA', 'La Liga', 'Primera División', 'Fútbol', 'Liga Española']}
